<a href="https://colab.research.google.com/github/Hiromi06/machine-translation/blob/main/train_on_chunk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [1]:
import os
import sys
import pprint
import torch
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from transformers import BitsAndBytesConfig
from tqdm import tqdm
import gc
#import bitsandbytes as bnb
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jul 22 14:08:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0              52W / 400W |   9059MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
"""
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
"""

In [4]:
from transformers import BertTokenizer, BertConfig, BertForMaskedLM, AdamW, get_linear_schedule_with_warmup

model_name = 'bert-base-multilingual-cased'
#device = torch.device(f'cuda:{rank}')
config = BertConfig.from_pretrained(model_name)
#bert_masked_model = BertForMaskedLM.from_pretrained(model_name, torch_dtype=torch.float16, low_cpu_mem_usage=True, quantization_config=nf4_config)
bert_masked_model = BertForMaskedLM.from_pretrained(model_name, config=config).to(device)
#bert_masked_model = DDP(bert_masked_model, device_ids=[rank])
optimizer = torch.optim.AdamW(bert_masked_model.parameters(), lr=2e-5)




config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
"""
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist

# Initialize the process group
dist.init_process_group(backend='nccl')

# Wrap the model with DDP
model = DDP(bert_masked_model)
"""

In [ ]:
"""

def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'  # non-used port number
    dist.init_process_group("nccl", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()

"""

In [5]:
class TranslationDataset(Dataset):
    def __init__(self, input_data, label_data):
        self.input_data = input_data
        self.label_data = label_data

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        #input_item = {key: val[idx] for key, val in self.input_data.items()}
        #label_item = {key: val[idx] for key, val in self.label_data.items()}
        input_item = self.input_data[idx]
        label_item = self.label_data[idx]
        return {
            'input_ids': input_item['input_ids'],
            'attention_mask': input_item['attention_mask'],
            'labels': label_item['input_ids']  # For translation, the label is the input_ids of the target language
        }


def get_file_paths(directory, prefix):
    return [os.path.join(directory, f) for f in sorted(os.listdir(directory)) if f.startswith(prefix) and f.endswith('.pt')]

def load_chunk(file_path):
    return torch.load(file_path)

def clear_memory():
    gc.collect()
    torch.cuda.ipc_collect()
    torch.cuda.empty_cache()

def train_on_chunk(bert_masked_model, optimizer, scheduler, device, input_chunk_path, label_chunk_path, batch_size=16):
    input_data = load_chunk(input_chunk_path)
    label_data = load_chunk(label_chunk_path)

    dataset = TranslationDataset(input_data, label_data)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    total_loss = 0
    bert_masked_model.train()
    scaler = GradScaler()

    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        input_ids = batch['input_ids'].squeeze(1).to(device)
        attention_masks = batch['attention_mask'].squeeze(1).to(device)
        labels = batch['labels'].squeeze(1).to(device)

        #print(f"input_ids shape: {input_ids.shape}")
        #print(f"attention_masks shape: {attention_masks.shape}")
        #print(f"labels shape: {labels.shape}")
        # without squeeze, the data looks like this
        # input_ids shape: torch.Size([16, 1, 512])
        # attention_masks shape: torch.Size([16, 1, 512])
        # labels shape: torch.Size([16, 1, 512])

        optimizer.zero_grad()


        # apply AMP (Mixed-Precision Training)
        with torch.cuda.amp.autocast():
          outputs = bert_masked_model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
          loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        """

        # Regular (Non-AMP) Training
        outputs = bert_masked_model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        """

        total_loss += loss.item()

        loop.set_postfix(loss=total_loss/len(dataloader))

        scheduler.step()

    avg_train_loss = total_loss / len(dataloader)
    print(f"Train Loss: {avg_train_loss}")


    # Clear memory
    del input_data, label_data, dataset, dataloader
    clear_memory()

def validate_on_chunks(bert_masked_model, device, val_input_files, val_label_files, batch_size=16):
    bert_masked_model.eval()
    total_eval_loss = 0

    for input_chunk_path, label_chunk_path in zip(val_input_files, val_label_files):
        input_data = load_chunk(input_chunk_path)
        label_data = load_chunk(label_chunk_path)

        dataset = TranslationDataset(input_data, label_data)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        with torch.no_grad():
            loop = tqdm(dataloader, leave=True)
            for batch in loop:
                input_ids = batch['input_ids'].squeeze(1).to(device)
                attention_masks = batch['attention_mask'].squeeze(1).to(device)
                labels = batch['labels'].squeeze(1).to(device)

                outputs = bert_masked_model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
                loss = outputs.loss
                total_eval_loss += loss.item()

                loop.set_postfix(loss=total_eval_loss/((i+1) * len(dataloader)))

        # Clear memory
        del input_data, label_data, dataset, dataloader
        clear_memory()

    avg_val_loss = total_eval_loss / len(val_input_files)
    print(f"Validation Loss: {avg_val_loss}")




# Prepare file paths for training and validation
encoded_data_dir = '/content/drive/MyDrive/machine_learning'
ja_train_files = get_file_paths(encoded_data_dir, 'ja_train_encoded_chunk')
en_train_files = get_file_paths(encoded_data_dir, 'en_train_encoded_chunk')
ja_test_files = get_file_paths(encoded_data_dir, 'ja_test_encoded_chunk')
en_test_files = get_file_paths(encoded_data_dir, 'en_test_encoded_chunk')


num_training_steps = 3 * len(ja_train_files)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Train the model one chunk at a time
for epoch in range(3):  # Number of epochs
    for i in range(len(ja_train_files)):
        print(f"Epoch {epoch + 1}, Chunk {i + 1}")
        train_on_chunk(bert_masked_model, optimizer, scheduler, device, ja_train_files[i], en_train_files[i])

    # Perform validation after each epoch
    print(f"Epoch {epoch + 1}, Validation")
    validate_on_chunks(bert_masked_model, device, ja_test_files, en_test_files)

    # Save the model at the end of each epoch
    model_save_path = f'/content/drive/MyDrive/machine_learning/bert_model_epoch_{epoch+1}.pt'
    bert_masked_model.save_pretrained(model_save_path)
    print(f"Model saved at {model_save_path}")

# Save the final model
final_model_save_path = '/content/drive/MyDrive/machine_learning/bert_model_final.pt'
bert_masked_model.save_pretrained(final_model_save_path)
print(f"Final model saved at {final_model_save_path}")

# Save the final model
#bert_masked_model.save_pretrained('/content/drive/MyDrive/machine_learning')

Epoch 1, Chunk 1


100%|██████████| 12325/12325 [32:52<00:00,  6.25it/s, loss=24.1]


Train Loss: 24.09885055155106
Epoch 1, Validation


100%|██████████| 1370/1370 [04:20<00:00,  5.26it/s, loss=26]


Validation Loss: 35645.84383010864
Model saved at /content/drive/MyDrive/machine_learning/bert_model_epoch_1.pt
Epoch 2, Chunk 1


...

Validation Loss: 35645.84383010864
Model saved at /content/drive/MyDrive/machine_learning/bert_model_epoch_2.pt
Epoch 3, Chunk 1


100%|██████████| 12325/12325 [32:55<00:00,  6.24it/s, loss=24.1]


Train Loss: 24.101598502385446
Epoch 3, Validation


100%|██████████| 1370/1370 [04:20<00:00,  5.25it/s, loss=26]


Validation Loss: 35645.84383010864
Model saved at /content/drive/MyDrive/machine_learning/bert_model_epoch_3.pt
Final model saved at /content/drive/MyDrive/machine_learning/bert_model_final.pt
